<a href="https://colab.research.google.com/github/pj747/qml-experiments/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
!pip install wandb
!wandb login

wandb: Currently logged in as: pj747 (use `wandb login --relogin` to force relogin)


In [1]:
!pip install pennylane --upgrade
!pip install pennylane-qulacs["gpu"] --upgrade

     |████████████████████████████████| 460kB 7.4MB/s 
     |████████████████████████████████| 215kB 9.8MB/s 
  Created wheel for qulacs-gpu: filename=Qulacs_GPU-0.2.0-cp37-cp37m-linux_x86_64.whl size=884819 sha256=9fae4bf4b32cb640f412985641656c7b1913cf5127eeccf026801f263e4f3aa4
  Stored in directory: /root/.cache/pip/wheels/dc/c6/90/6f34fb74a323135ff90fa1a844f9fd5cbc5c6d98edaf9c347f
Successfully built qulacs-gpu


In [2]:
import pennylane as qml
from pennylane import qnn

In [4]:
import math
from pennylane import numpy as np
from types import SimpleNamespace
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
tf.keras.backend.set_floatx('float64')

In [16]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def qnode(inputs, weights_1, weights_2):
    for i in range(0,len(inputs)-1,2): 
        qml.RX(inputs[i], wires=[0])
    qml.CZ(wires=[1,0])
    qml.Rot(*weights_1, wires=0)
    qml.Rot(*weights_2, wires=1)
    return (qml.expval(qml.PauliZ(0)))

In [17]:
weight_shapes = {"weights_1": 3, "weights_2" : 3}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=1)
clayer = tf.keras.layers.Dense(5, input_dim=30, activation='relu')
outLayer = tf.keras.layers.Dense(1, activation="sigmoid")
model = tf.keras.models.Sequential([clayer, qlayer])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 5)                 155       
_________________________________________________________________
keras_layer_1 (KerasLayer)   (None, 1)                 0 (unused)
Total params: 155
Trainable params: 155
Non-trainable params: 0
_________________________________________________________________


In [18]:
dataSet = load_breast_cancer()
X = dataSet.data
Y = dataSet.target
Y = Y * 2 - np.ones(len(Y))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=1)

512/512 [==============================] - 36s 71ms/step - loss: 3.0058 - accuracy: 0.1797


In [19]:
model.predict(X_test)

array([ 0.10618519,  0.11907121,  0.74170135, -0.83247078, -0.86745871,
       -0.89731569, -0.61598684, -0.88546966, -0.12620447, -0.03239429,
       -0.2901519 ,  0.70704149, -0.10504295,  0.9027526 , -0.11649201,
        0.89051924, -0.81087887,  0.90836407,  0.88090455, -0.08907565,
       -0.86919828,  0.89845098,  0.90089307, -0.30083666,  0.89514369,
       -0.88798603,  0.90860782,  0.13106108,  0.67407019, -0.80488146,
        0.22603213,  0.90755108,  0.67985728, -0.50504577, -0.86388475,
       -0.3850783 , -0.0447228 , -0.02968343, -0.63170397, -0.155899  ,
        0.85323445, -0.76819225,  0.80305477,  0.41444482, -0.40076743,
        0.22983356,  0.59189156, -0.60693799, -0.30872949,  0.78437931,
       -0.80141832,  0.90762561,  0.76256982, -0.71633777,  0.59126142,
        0.90645841, -0.60531354])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 155       
_________________________________________________________________
keras_layer (KerasLayer)     (None, 2)                 6         
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import sklearn.datasets
data = sklearn.datasets.make_moons()
X = tf.constant(data[0])
Y = tf.one_hot(data[1], depth=2)

opt = tf.keras.optimizers.SGD(learning_rate=0.5)
model.compile(opt, loss='mae', metrics=['accuracy'])

model.fit(X, Y, epochs=8, batch_size=5)
model.summary()

In [5]:
def makeModel(config):
    numQubits = config.numQubits
    dev = qml.device("qulacs.simulator", wires=numQubits, shots=1000, gpu=True)
    @qml.qnode(dev)
    def qcircuit(inputs, param):
        for i in range(config.numLayers):
            # norm = np.linalg.norm(inputs)
            # norm = norm if norm !=0 else 1 
            for i in range(0, len(inputs)-numQubits, numQubits):
                for j in range(numQubits):
                    qml.RX(inputs[i+j]*2*math.pi/norm, wires=j)
            if config.fullEntangle:
                for j in range(numQubits):
                    for i in range(j):
                        qml.CZ(wires=[j,i])
            else:
                for j in range(numQubits-1):
                    qml.CZ(wires=[j,j+1])
            for j in range(numQubits):
                qml.Rot(param[j][i][0], param[j][i][1], param[j][i][2], wires = [j])
            
        return [qml.expval(qml.PauliZ(0))]
        
    
    return qcircuit

In [98]:
config = SimpleNamespace(
        numQubits = 1,
        numLayers = 1,
        dev = "default.qubit",
        batchSize = 4,
        numBatches = 300,
        fullEntangle = True
)
node = makeModel(config)
weight_shapes = {"param": (config.numQubits, config.numLayers, 3)}


In [99]:
qlayer = qml.qnn.KerasLayer(node, weight_shapes, output_dim=1)

In [100]:
inputLayer = K.layers.InputLayer(30)
clayer = Dense(30)
model = Sequential([inputLayer, qlayer])
model.compile(Adam(), loss = 'mse', metrics=['accuracy'])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_15 (KerasLayer)  (None, 1)                 0 (unused)
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [102]:
print(X_train.shape)

(512, 30)


In [103]:
fitting = model.fit(X_train, Y_train, epochs=6, batch_size=config.batchSize, verbose=2)

Epoch 1/6


InvalidArgumentError: ignored

In [ ]:
def accuracy(labels, predictions):
    accuracy = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            accuracy = accuracy + 1
    accuracy = accuracy / len(labels)

    return accuracy

def lossFunction(labels, predictedLabels):
    loss = 0
    for l, p in zip(labels, predictedLabels):
        loss += (l-p) ** 2
    loss /= len(labels)
    return loss

def cost(qcircuit, params, X, Y):
    predictions = [getPrediction(qcircuit, params, x) for x in X]
    return lossFunction(Y, predictions)

def getPrediction(qcircuit, params, data=None):
    
    quantumOutput = qcircuit(params, data)
    # print("For", data)
    # print("Prediction: ", quantumOutput)
    return quantumOutput
    #no bias

def makeModel(config):
    numQubits = config.numQubits
    dev = qml.device("qulacs.simulator", wires=numQubits, shots=1000, gpu=True)
    @qml.qnode(dev)
    def qcircuit(param, data):
        for i in range(config.numLayers):
            embeddingCircuit(config, data)
            if config.fullEntangle:
                for j in range(numQubits):
                    for i in range(j):
                        qml.CZ(wires=[j,i])
            else:
                for j in range(numQubits-1):
                    qml.CZ(wires=[j,j+1])
            for j in range(numQubits):
                qml.Rot(param[j][i][0], param[j][i][1], param[j][i][2], wires = [j])
            
        return qml.expval(qml.PauliZ(0))

    def embeddingCircuit(config, data):
        norm = np.linalg.norm(data)
        norm = norm if norm !=0 else 1 
        for i in range(0, len(data)-numQubits, numQubits):
            for j in range(numQubits):
                qml.RX(data[i+j]*2*math.pi/norm, wires=j)
    
    return qcircuit


def createAndTrain(config, WandB = False):
    varInit = 0.01 * np.random.randn(config.numQubits, config.numLayers, 3)
    opt = qml.AdamOptimizer()
    batchSize = config.batchSize
    var = varInit
    qcircuit = makeModel(config)
    qcircuit(var, X_train[0])
    circ = qcircuit.draw()
    if WandB:
        wandb.log({"circuit": str(circ)})

    print("Sample Circuit:\n" , circ)
    numBatches = config.numBatches
    for it in range(numBatches):
        batchIndex = np.random.randint(0, len(X_train), (batchSize,))
        X_batch = X_train[batchIndex]
        Y_batch = Y_train[batchIndex]
        var = opt.step(lambda v: cost(qcircuit, v, X_batch, Y_batch), var)
        # Compute accuracy
        print("Computed batch ", it, var)
        predictions = []
        if (it%10 == 0):
            for x in X_test:
                #print(var, x)
                op = qcircuit(var, x)
                predictions.append(np.sign(op))

                #print(qcircuit.draw())
            acc = accuracy(Y_test, predictions)
            loss = lossFunction(Y_test, predictions)
            test_loss = float(loss)
            test_acc = float(acc)
            predictions = []
            if (it%50==0):
                for x in X_train:
                    op = qcircuit(var, x)
                    predictions.append(np.sign(op))
                acc = accuracy(Y_train, predictions)
                loss = lossFunction(Y_train, predictions)
                loss = float(loss)
                a = float(acc)
                if WandB:
                    wandb.log({"train_cost":loss, "train_acc":a}, commit=False)
            if WandB:
                wandb.log({"test_cost":test_loss, "test_acc":test_acc})
    if WandB:
        wandb.log({"parameters": var})
    wandb.finish()

In [ ]:
def wandbRun(config):
    wandb.init(project='qml-experiments', entity='pj747', config=config)
    createAndTrain(config, WandB=True)

def wandbSweep():
    run = wandb.init()
    config = run.config
    createAndTrain(config, WandB=True)